In [0]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [2]:
# fix random seed so we can get the same result
np.random.seed(7)

# load the dataset

# only want to retrieve the most frequent 5000 words
num_words = 5000 
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)
np.load = np_load_old

# truncate and pad input sequences to the same length

# The max input size is 500
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)


17465344/17464789 [==============================] - 0s 0us/step


In [3]:
print('Train data shape:', X_train.shape)
print('Test data shape:', X_test.shape)

# you can print out the first example to see what it looks like

print('The first training example:')
print(X_train[0])
print('The first training label:')
print(y_train[0])

Train data shape: (25000, 500)
Test data shape: (25000, 500)
The first training example:
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    

In [0]:
'''
P1
We want to randomly

- pick 5000 instances from training data
- pick 1000 instances from testing data

'''
train_indexes = np.random.choice(len(X_train), 5000, replace=False)
test_indexes = np.random.choice(len(X_test), 1000, replace=False)
X_train = X_train[train_indexes]
y_train = y_train[train_indexes]
X_test = X_test[test_indexes]
y_test = y_test[test_indexes]

In [5]:
'''
P2

In this section, we want to construct a LSTM model

Outline:
sequence of words -> word embeddings(input) -> dropout 20% of the input ->
LSTM of 100 units -> dropout 20% of the output -> sigmoid activation -> 0 or 1

complete the model graph
- add input dropout
- add lstm layer
- add output dropout
- use sigmoid activation
- use binary_crossentropy loss function

'''

# create the model
vec_len = 32
model = Sequential()
model.add(Embedding(num_words, vec_len, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0706 19:11:40.702961 139721036928896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 19:11:40.737390 139721036928896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 19:11:40.744193 139721036928896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0706 19:11:40.858706 139721036928896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0706 19:11:40.874373 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
'''
P3
- train 3 epochs with batch size of 64
- do evaluation on testing data

'''

# training and evaluation
model.fit(X_train, y_train, epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)

Epoch 1/3
5000/5000 [==============================] - 69s 14ms/step - loss: 0.6726 - acc: 0.5880
Epoch 2/3
5000/5000 [==============================] - 64s 13ms/step - loss: 0.4545 - acc: 0.8006
Epoch 3/3
5000/5000 [==============================] - 64s 13ms/step - loss: 0.3331 - acc: 0.8646


In [0]:
print("Accuracy: %.2f%%" % (scores[1]*100))

In [9]:
# create the model
vec_len = 32
model = Sequential()
model.add(Embedding(num_words, vec_len, input_length=max_review_length))
model.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
scores = model.evaluate(X_test, y_test, verbose=0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 100)               13300     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 173,401
Trainable params: 173,401
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
5000/5000 [==============================] - 23s 5ms/step - loss: 0.7154 - acc: 0.5004
Epoch 2/3
5000/5000 [==============================] - 22s 4ms/step - loss: 0.6996 - acc: 0.5208
Epoch 3/3
5000/5000 [==============================] - 22s 4ms/step - loss: 0.7032 - acc: 0.5176


In [10]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 47.30%
